# Toggle switch

In [1]:
from scipy.integrate import solve_ivp

from ode_helper import *

## Simulation parameters

In [2]:
t = 500
n = np.array([51, 51], dtype="int64")
r = 5
m = n.size
dx = np.prod(n)

## Reaction system

In [3]:
# Reaction parameters
b = 0.4
c = 0.05

# Stoichiometric vectors
nu0 = np.array([-1, 0])
nu1 = np.array([0, -1])
nu2 = np.array([1, 0])
nu3 = np.array([0, 1])

# Propensity functions
@njit
def prop0(x: np.ndarray) -> float:
    return c * x[0]
@njit
def prop1(x: np.ndarray) -> float:
    return c * x[1]
@njit
def prop2(x: np.ndarray) -> float:
    return b / (b + x[1])
@njit
def prop3(x: np.ndarray) -> float:
    return b / (b + x[0])

## RHS of the CME

In [4]:
@njit
def cme(t: float, P: np.ndarray, interval: np.ndarray) -> np.ndarray:
    m = interval.size
    null = np.zeros(m)
    result = (
        EvaluateProp(prop0, nu0, interval) * ShiftArray(P, nu0, interval) +
        EvaluateProp(prop1, nu1, interval) * ShiftArray(P, nu1, interval) +
        EvaluateProp(prop2, nu2, interval) * ShiftArray(P, nu2, interval) +
        EvaluateProp(prop3, nu3, interval) * ShiftArray(P, nu3, interval)
    )
    result -= (
        EvaluateProp(prop0, null, interval) +
        EvaluateProp(prop1, null, interval) +
        EvaluateProp(prop2, null, interval) +
        EvaluateProp(prop3, null, interval)
    ) * P
    return result

## Set up the initial condition

In [5]:
C = 0.5 * np.array([[75, -15], [-15, 75]])
Cinv = np.linalg.inv(C)
mu = np.array([30, 5])

def eval_P0(x: np.ndarray) -> float:
    return np.exp(-0.5 * np.dot(np.transpose(x - mu), np.dot(Cinv, (x - mu))))

P0 = ConstructP0(eval_P0, n)

## Solve the system

In [6]:
t_step = 10
t_eval = np.arange(0, t + t_step, t_step)
sol = solve_ivp(lambda t, P: cme(t, P, n), [
                0, t + 1], P0, method='RK45', t_eval=t_eval)

y = np.zeros((t_eval.size, dx))
for i in range(t_eval.size):
    y[i, :] = sol.y[:, i]

## Calculate output

In [7]:
P_marginal, P_marginal2D, P_sliced, P_sliced2D, P_best_approximation = CalculateObservables(
    y, n, r, np.array([0, 1]), 1)

with open("scripts/reference_solutions/ts_ode_ref.npy", "wb") as f:
    np.save(f, P_marginal)
    np.save(f, P_marginal2D)
    np.save(f, P_sliced)
    np.save(f, P_sliced2D)
    np.save(f, P_best_approximation)
    np.save(f, n)